# HNSW优化测试

本notebook用于测试HNSW优化的两个主要功能：
1. 高层桥边（High-layer Bridge Edges）
2. 自适应多入口种子（Adaptive Multi-entry Seeds）

使用Text2Image数据集进行测试，并与基线HNSW进行对比。


In [60]:
# 导入必要的库
import sys
import os
sys.path.append('/root/code/vectordbindexing')
sys.path.append('/root/code/vectordbindexing/hnsw_optimization')

import numpy as np
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from io_utils import read_fbin, read_ibin
import faiss

# 导入我们的优化模块
from data_loader import DataLoader
from gt_utils import GroundTruthComputer
from hnsw_baseline import HNSWBaseline
from bridge_builder import BridgeBuilder
from multi_entry_search import MultiEntrySearch, AdaptiveMultiEntrySearch
from hnsw_with_bridges import HNSWWithBridges  # 新增：集成版 HNSW

print("所有模块导入成功！")

所有模块导入成功！


## 1. 数据加载

加载Text2Image数据集和预计算的ground truth


In [59]:
# 数据路径
file_path = "/root/code/vectordbindexing/Text2Image/base.1M.fbin"
query_path = "/root/code/vectordbindexing/Text2Image/query.public.100K.fbin"
faiss_top100_path = "/root/code/vectordbindexing/faiss_top100_results.json"
faiss_stats_path = "/root/code/vectordbindexing/faiss_effort_stats.json"
faiss_effort_perc = "/root/code/vectordbindexing/faiss_effort_percentiles.json"

print("加载数据集...")

# 读取数据集
data_vector = read_fbin(file_path)
query_vector = read_fbin(query_path)

print(f"数据向量形状: {data_vector.shape}")
print(f"查询向量形状: {query_vector.shape}")

# 为了测试，使用较小的数据集
n_train = len(data_vector)  # 训练数据
n_query = len(query_vector)  # 查询数据

X = data_vector[:n_train]
Q = query_vector[:n_query]

print(f"\n使用训练数据: {X.shape}")
print(f"使用查询数据: {Q.shape}")

# 加载 FAISS ground truth 结果
print("加载 FAISS ground truth 结果...")
gt_computer_loader = GroundTruthComputer()
gt_neighbors = gt_computer_loader.load_ground_truth_from_json(faiss_top100_path, n_queries=n_query, k=100)
print(f"Ground truth 形状: {gt_neighbors.shape}")

np.random.seed(42)


加载数据集...


数据向量形状: (1000000, 200)
查询向量形状: (100000, 200)

使用训练数据: (1000000, 200)
使用查询数据: (100000, 200)
加载 FAISS ground truth 结果...
Ground truth 形状: (100000, 100)


## 2.5. 集成版 HNSW：自动桥接边 + 多入口搜索

使用新的 `HNSWWithBridges` 类，将桥接边构建和多入口搜索直接集成到 HNSW 中。

**关键特性**：
- 构建时自动添加桥接边（基于2跳可达性检测）
- 搜索时自动使用多入口点
- 单一接口，无需手动管理多个组件


In [ ]:
print("=" * 70)
print("构建集成版 HNSW（自动添加桥接边 + 多入口搜索）")
print("=" * 70)

# 创建集成版 HNSW
hnsw_integrated = HNSWWithBridges(
    dimension=X.shape[1],
    M=64,
    ef_construction=200,
    # 桥接边配置
    enable_bridges=True,
    bridge_sample_ratio=0.05,  # 5% 采样（500K数据用较小比例）
    max_hop_distance=2,         # 检查2跳可达性
    # 多入口搜索配置
    enable_multi_entry=True,
    num_entry_points=4
)

# 构建索引（自动添加桥接边）
start_time = time.time()
hnsw_integrated.build_index(X)
integrated_build_time = time.time() - start_time

print(f"\n✅ 构建完成，耗时: {integrated_build_time:.2f}秒")

# 统计信息
stats = hnsw_integrated.get_statistics()
print(f"\n统计信息:")
print(f"  总节点数: {stats['total_nodes']}")
print(f"  桥接边数: {stats['total_bridges']} 条")
print(f"  桥接边密度: {stats['total_bridges']/stats['total_nodes']:.4f}")
print(f"  高层节点分布:")
for layer in sorted(stats['high_layer_count'].keys(), reverse=True):
    count = stats['high_layer_count'][layer]
    print(f"    Layer {layer}: {count} 个节点")

# 测试单个查询
print("\n测试集成版搜索...")
test_query = Q[0]
integrated_neighbors, integrated_cost = hnsw_integrated.search(
    test_query, k=100, ef_search=200
)

print(f"搜索结果: {len(integrated_neighbors)} 个邻居，成本: {integrated_cost}")
print(f"前10个邻居: {integrated_neighbors[:10]}")


构建集成版 HNSW（自动添加桥接边 + 多入口搜索）


In [ ]:
print("测试不同的入口点数量（num_entry_points）")
print("=" * 70)

# 使用前100个查询进行测试
n_test_queries = min(100, len(Q))
print(f"测试查询数: {n_test_queries}")

# 准备 ground truth - 使用已加载的 gt_neighbors
gt_eval = GroundTruthComputer()
gt_eval.gt_neighbors = gt_neighbors[:n_test_queries]  # 关键：设置 ground truth

print(f"Ground truth 设置完成: {gt_eval.gt_neighbors.shape}")

# 测试不同的入口点数量
entry_point_values = [1, 2, 4, 8]  # 测试1, 2, 4, 8个入口点
results_by_entry = {}

for num_entries in entry_point_values:
    print(f"\n测试 num_entry_points={num_entries}...")
    
    # 创建 HNSW
    hnsw_test = HNSWWithBridges(
        dimension=X.shape[1],
        M=64,
        ef_construction=200,
        enable_bridges=True,
        bridge_sample_ratio=0.05,
        max_hop_distance=2,
        enable_multi_entry=(num_entries > 1),  # num_entries=1时禁用多入口
        num_entry_points=num_entries
    )
    
    # 构建索引
    build_start = time.time()
    hnsw_test.build_index(X)
    build_time_test = time.time() - build_start
    
    # 批量搜索
    all_neighbors = []
    search_times = []
    
    for j in range(n_test_queries):
        if j % 25 == 0:
            print(f"  处理查询 {j+1}/{n_test_queries}...")
        
        start = time.time()
        neighbors, _ = hnsw_test.search(Q[j], k=100, ef_search=200)
        search_times.append(time.time() - start)
        all_neighbors.append(neighbors)
    
    all_neighbors = np.array(all_neighbors)
    
    # 计算 recall（使用相同的 ground truth）
    recall_10 = gt_eval.compute_recall(all_neighbors, k_eval=10)
    recall_100 = gt_eval.compute_recall(all_neighbors, k_eval=100)
    avg_time = np.mean(search_times) * 1000
    
    results_by_entry[num_entries] = {
        'recall_10': recall_10,
        'recall_100': recall_100,
        'avg_time_ms': avg_time,
        'build_time': build_time_test,
        'bridges': hnsw_test.get_statistics()['total_bridges']
    }
    
    print(f"  构建时间: {build_time_test:.2f}s")
    print(f"  Recall@10:  {recall_10:.4f}")
    print(f"  Recall@100: {recall_100:.4f}")
    print(f"  查询时间: {avg_time:.3f}ms")
    print(f"  桥接边: {results_by_entry[num_entries]['bridges']} 条")

# 显示对比结果
print("\n" + "=" * 70)
print("不同入口点数量的性能对比")
print("=" * 70)
print(f"\n{'入口点数':<10} {'Recall@10':<12} {'Recall@100':<12} {'查询时间(ms)':<15} {'桥接边':<10}")
print("-" * 60)

for num_entries in entry_point_values:
    result = results_by_entry[num_entries]
    print(f"{num_entries:<10} {result['recall_10']:<12.4f} {result['recall_100']:<12.4f} {result['avg_time_ms']:<15.3f} {result['bridges']:<10}")

# 分析最佳配置
best_entry = max(entry_point_values, key=lambda x: results_by_entry[x]['recall_10'])
print(f"\n💡 分析:")
print(f"  最佳入口点数: {best_entry} (Recall@10={results_by_entry[best_entry]['recall_10']:.4f})")

print(f"\n✅ 入口点数量测试完成")


测试不同的入口点数量（num_entry_points）
测试查询数: 100
Ground truth 设置完成: (100, 100)

测试 num_entry_points=1...
